# Modeling Example: The Blog Application

Modeling an application is an iterative process.
It involves the use of several fundamental principles and patterns.
This document walks through that process using the familiar blog as an example.

In [1]:
// Reference the Jinaga NuGet packages
#r "nuget: Jinaga, 0.11.18"
#r "nuget: Jinaga.Graphviz, 0.11.18"
#r "nuget: Jinaga.UnitTest, 0.11.18"

Installed Packages Jinaga, 0.11.18 Jinaga.Graphviz, 0.11.18 Jinaga.UnitTest, 0.11.18

In [2]:
using Jinaga;
using Jinaga.Graphviz;
using Jinaga.UnitTest;

// Create a Jinaga client for unit testing
var jinagaClient = JinagaTest.Create(opt =>
{
    // Simulate a logged in user
    opt.User = new User("--- FAKE USER ---");
});

## Declaring Facts

Express a Jinaga model as a set of C# records representing historical facts.
Decorate each record with a `FactType` attribute.
The fields of the records are predecessors -- facts that came before -- and fields.

In the Blog model, a site is an entity created by a specific user.
We use the date and time to distinguish it from other sites that the user created.
The assumption is that a user will not create multiple sites within the same millisecond.

Notice that the site does not contain any other properties, such as a title or a domain name.
Those properties are mutable, and therefore not part of the initial fact.

In [3]:
[FactType("Blog.Site")]
public record Site(User creator, DateTime createdAt) { }

The `User` type is provided by the Jinaga library.
It represents someone who can log into the app.
A model almost always starts with a fact owned by a user.
This gives us a place to start our authorization rules.

In [4]:
// Call RenderTypes without a semicolon to display the graph
Renderer.RenderTypes(typeof(User), typeof(Site))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator

## Log In

The user is the starting point of the model.
You'll need to call `Login` to get the logged in user.

In [5]:
// Login returns the user fact and profile information
var (user, profile) = await jinagaClient.Login();

// Call RenderFacts on the Jinaga client to display the facts
jinagaClient.RenderFacts(user)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER ---

## Save Facts

Save an instance of a fact to the Jinaga client whenever the user takes an action.
Pass a record to the `Fact` method.
This will save the fact in the local store and share it with the server.

The `Fact` method returns the record that was just saved.
Hold on to this copy of the record for use in other methods.

In [6]:
var site = await jinagaClient.Fact(new Site(user, DateTime.UtcNow));

jinagaClient.RenderFacts(site)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T14:41:14.... 
 
 
<!-- 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator

## Query for Successors

Given one fact, you can find its successors using LINQ.
For example, you can find all sites for a given user.
Use `Given<T>.Match()` to define a specification.

In [7]:
// The parameters to the lambda are the given fact (in this case, the user) and
// the fact repository. Use the OfType<T> method to get facts from the repository.
var sitesByUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user
    select site);

// Query for facts matching the specification.
var sites = await jinagaClient.Query(sitesByUser, user);

jinagaClient.RenderFacts(sites)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T14:41:14.... 
 
 
<!-- 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator

## Deleting Facts

Facts are not truly deleted.
You indicate that they should be deleted by introducing a new fact.
There are mechanisms for taking them out of storage, but we'll cover those later.

To indicate that a site should be deleted, define a fact that refers to the site as a predecessor.
It also should have a timestamp so that we can differentiate one deletion from another.

In [8]:
[FactType("Blog.Site.Deleted")]
public record SiteDeleted(Site site, DateTime deletedAt) { }

Renderer.RenderTypes(typeof(SiteDeleted))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Site.Deleted -->
 
 Blog.Site.Deleted 
 
 Blog.Site.Deleted 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Blog.Site.Deleted->Blog.Site -->
 
 Blog.Site.Deleted->Blog.Site 
 
 
 site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator

To indicate that a site should be deleted, create an instance of that fact.

In [9]:
var siteDeleted = await jinagaClient.Fact(new SiteDeleted(site, DateTime.UtcNow));

jinagaClient.RenderFacts(siteDeleted)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T14:41:14.... 
 
<!-- 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- CzZ6vsrLSzz25s9F0cATkxibFf+PCv6+NBoW5iWZ8gNJ7j84qGakgJl/hrHkE9VSqkQuKMTjMKA0xgoC9gqYjA== -->
 
 CzZ6vsrLSzz25s9F0cATkxibFf+PCv6+NBoW5iWZ8gNJ7j84qGakgJl/hrHkE9VSqkQuKMTjMKA0xgoC9gqYjA== 
 
 Blog.Site.Deleted 
 
 deletedAt 
 
 2024-05-18T14:46:22.... 
 
 
<!-- CzZ6vsrLSzz25s9F0cATkxibFf+PCv6+NBoW5iWZ8gNJ7j84qGakgJl/hrHkE9VSqkQuKMTjMKA0xgoC9gqYjA==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 CzZ6vsrLSzz25s9F0cATkxibFf+PCv6+NBoW5iWZ8gNJ7j84qGakgJl/hrHkE9VSqkQuKMTjMKA0xgoC9gqYjA==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 
 site

If you query the specification now, you will still see the site.

In [10]:
sites = await jinagaClient.Query(sitesByUser, user);

sites.Count()

1

That's because we need to change the specification to exclude deleted sites.
Add a clause that filters out sites that have a site deleted successor.

In [11]:
sitesByUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user
    // Include only the sites that have not been deleted
    where !facts.Any<SiteDeleted>(sd => sd.site == site)
    select site);

sites = await jinagaClient.Query(sitesByUser, user);

sites.Count()

0

## Restoring Facts

Users sometimes make mistakes.
They should be able to undo a deletion.
To allow this, create a new fact type that represents the restoration.

To restore a site, define a fact type that refers to the deletion of that site.
It needs no additional parameters.

In [13]:
[FactType("Blog.Site.Restored")]
public record SiteRestored(SiteDeleted deleted) { }

Renderer.RenderTypes(typeof(SiteRestored))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Site.Restored -->
 
 Blog.Site.Restored 
 
 Blog.Site.Restored 
 
<!-- Blog.Site.Deleted -->
 
 Blog.Site.Deleted 
 
 Blog.Site.Deleted 
 
<!-- Blog.Site.Restored->Blog.Site.Deleted -->
 
 Blog.Site.Restored->Blog.Site.Deleted 
 
 
 deleted 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Blog.Site.Deleted->Blog.Site -->
 
 Blog.Site.Deleted->Blog.Site 
 
 
 site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator

Create an instance of this fact to indicate that the deletion should no longer take effect.

In [14]:
var siteRestored = await jinagaClient.Fact(new SiteRestored(siteDeleted));

jinagaClient.RenderFacts(siteRestored)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T14:41:14.... 
 
<!-- 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- CzZ6vsrLSzz25s9F0cATkxibFf+PCv6+NBoW5iWZ8gNJ7j84qGakgJl/hrHkE9VSqkQuKMTjMKA0xgoC9gqYjA== -->
 
 CzZ6vsrLSzz25s9F0cATkxibFf+PCv6+NBoW5iWZ8gNJ7j84qGakgJl/hrHkE9VSqkQuKMTjMKA0xgoC9gqYjA== 
 
 Blog.Site.Deleted 
 
 deletedAt 
 
 2024-05-18T14:46:22.... 
 
<!-- CzZ6vsrLSzz25s9F0cATkxibFf+PCv6+NBoW5iWZ8gNJ7j84qGakgJl/hrHkE9VSqkQuKMTjMKA0xgoC9gqYjA==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 CzZ6vsrLSzz25s9F0cATkxibFf+PCv6+NBoW5iWZ8gNJ7j84qGakgJl/hrHkE9VSqkQuKMTjMKA0xgoC9gqYjA==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 
 site 
 
<!-- 5Ku1FTgWpNtXxGVT9zXgr2jyxIYsTQmEvLPsVb7EvlUAfSs50/IRJJToM357IqJLnTkfp6UP6WMcaUu/BtOAVw== -->
 
 5Ku1FTgWpNtXxGVT9zXgr2jyxIYsTQmEvLPsVb7EvlUAfSs50/IRJJToM357IqJLnTkfp6UP6WMcaUu/BtOAVw== 
 
 Blog.Site.Restored 
 
 
<!-- 5Ku1FTgWpNtXxGVT9zXgr2jyxIYsTQmEvLPsVb7EvlUAfSs50/IRJJToM357IqJLnTkfp6UP6WMcaUu/BtOAVw==->CzZ6vsrLSzz25s9F0cATkxibFf+PCv6+NBoW5iWZ8gNJ7j84qGakgJl/hrHkE9VSqkQuKMTjMKA0xgoC9gqYjA== -->
 
 5Ku1FTgWpNtXxGVT9zXgr2jyxIYsTQmEvLPsVb7EvlUAfSs50/IRJJToM357IqJLnTkfp6UP6WMcaUu/BtOAVw==->CzZ6vsrLSzz25s9F0cATkxibFf+PCv6+NBoW5iWZ8gNJ7j84qGakgJl/hrHkE9VSqkQuKMTjMKA0xgoC9gqYjA== 
 
 
 deleted

As you might imagine, the specification doesn't honor this new fact yet.

In [15]:
sites = await jinagaClient.Query(sitesByUser, user);

sites.Count()

0

But if we filter the deletions to include only those that don't have a successor restore fact, then we get the desired behavior.

In [17]:
sitesByUser = Given<User>.Match((user, facts) =>
    from site in facts.OfType<Site>()
    where site.creator == user
    where !facts.Any<SiteDeleted>(
        sd => sd.site == site &&
        // Honor only the site deletion that have not been restored
        !facts.Any<SiteRestored>(sr => sr.deleted == sd))
    select site);

sites = await jinagaClient.Query(sitesByUser, user);

sites.Count()

1

## Mutable Properties

Facts are immutable.
But sometimes we want to record the values of properties that can change over time.
To do so, define a new fact type representing a change to that value.

The name of a site should be allowed to change.
That is why we didn't include it as a field of the `Site` fact.
To model this, define a `SiteName` fact that refers to the site and stores the new value.
It should also refer to past `SiteName` facts that it replaces.

In [18]:
[FactType("Blog.Site.Name")]
public record SiteName(Site site, string value, SiteName[] prior) { }

Renderer.RenderTypes(typeof(SiteName))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Blog.Site.Name -->
 
 Blog.Site.Name 
 
 Blog.Site.Name 
 
<!-- Blog.Site.Name->Blog.Site.Name -->
 
 Blog.Site.Name->Blog.Site.Name 
 
 
 prior 
 
<!-- Blog.Site -->
 
 Blog.Site 
 
 Blog.Site 
 
<!-- Blog.Site.Name->Blog.Site -->
 
 Blog.Site.Name->Blog.Site 
 
 
 site 
 
<!-- Jinaga.User -->
 
 Jinaga.User 
 
 Jinaga.User 
 
<!-- Blog.Site->Jinaga.User -->
 
 Blog.Site->Jinaga.User 
 
 
 creator

When we create the first instance of the site name fact, we have no prior names to replace.
So we pass in an empty array.

In [20]:
var siteName0 = await jinagaClient.Fact(new SiteName(site, "My Site", []));

jinagaClient.RenderFacts(siteName0)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T14:41:14.... 
 
<!-- 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- 0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w== -->
 
 0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w== 
 
 Blog.Site.Name 
 
 value 
 
 My Site 
 
 
<!-- 0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 
 site

If the user changes the name of the site, then we record that with a new fact that replaces the first one.

In [21]:
var siteName1 = await jinagaClient.Fact(new SiteName(site, "My Blog", [siteName0]));

jinagaClient.RenderFacts(siteName1)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T14:41:14.... 
 
<!-- 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- 0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w== -->
 
 0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w== 
 
 Blog.Site.Name 
 
 value 
 
 My Site 
 
<!-- 0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 
 site 
 
<!-- FDFCHhtkO3OemRIzHw33P4kgHee3S+ygHBLHkm1cQhLoID9W8OgvLBaSAdy4aJd+y3zsF1SIUrotWe4PKs2g1Q== -->
 
 FDFCHhtkO3OemRIzHw33P4kgHee3S+ygHBLHkm1cQhLoID9W8OgvLBaSAdy4aJd+y3zsF1SIUrotWe4PKs2g1Q== 
 
 Blog.Site.Name 
 
 value 
 
 My Blog 
 
 
<!-- FDFCHhtkO3OemRIzHw33P4kgHee3S+ygHBLHkm1cQhLoID9W8OgvLBaSAdy4aJd+y3zsF1SIUrotWe4PKs2g1Q==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 FDFCHhtkO3OemRIzHw33P4kgHee3S+ygHBLHkm1cQhLoID9W8OgvLBaSAdy4aJd+y3zsF1SIUrotWe4PKs2g1Q==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 
 site 
 
<!-- FDFCHhtkO3OemRIzHw33P4kgHee3S+ygHBLHkm1cQhLoID9W8OgvLBaSAdy4aJd+y3zsF1SIUrotWe4PKs2g1Q==->0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w== -->
 
 FDFCHhtkO3OemRIzHw33P4kgHee3S+ygHBLHkm1cQhLoID9W8OgvLBaSAdy4aJd+y3zsF1SIUrotWe4PKs2g1Q==->0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w== 
 
 
 prior

If they change it again, we only include the most recent value that we are replacing.
There is no need to list all of the past values, since some of them have already been replaced.
This forms a chain of values that the property took on over time.

In [22]:
var siteName2 = await jinagaClient.Fact(new SiteName(site, "My Journal", [siteName1]));

jinagaClient.RenderFacts(siteName2)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 Jinaga.User 
 
 publicKey 
 
 --- FAKE USER --- 
 
<!-- 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 Blog.Site 
 
 createdAt 
 
 2024-05-18T14:41:14.... 
 
<!-- 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== -->
 
 5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ==->eLG0D2gojV+51Ix80JNo2Uh4EQTKvjbZUq5JNrf9K5gSXEqRmi7LBSleOl09F/bWdrlWUgDkalEH847v83U7fA== 
 
 
 creator 
 
<!-- 0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w== -->
 
 0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w== 
 
 Blog.Site.Name 
 
 value 
 
 My Site 
 
<!-- 0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 
 site 
 
<!-- FDFCHhtkO3OemRIzHw33P4kgHee3S+ygHBLHkm1cQhLoID9W8OgvLBaSAdy4aJd+y3zsF1SIUrotWe4PKs2g1Q== -->
 
 FDFCHhtkO3OemRIzHw33P4kgHee3S+ygHBLHkm1cQhLoID9W8OgvLBaSAdy4aJd+y3zsF1SIUrotWe4PKs2g1Q== 
 
 Blog.Site.Name 
 
 value 
 
 My Blog 
 
<!-- FDFCHhtkO3OemRIzHw33P4kgHee3S+ygHBLHkm1cQhLoID9W8OgvLBaSAdy4aJd+y3zsF1SIUrotWe4PKs2g1Q==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 FDFCHhtkO3OemRIzHw33P4kgHee3S+ygHBLHkm1cQhLoID9W8OgvLBaSAdy4aJd+y3zsF1SIUrotWe4PKs2g1Q==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 
 site 
 
<!-- FDFCHhtkO3OemRIzHw33P4kgHee3S+ygHBLHkm1cQhLoID9W8OgvLBaSAdy4aJd+y3zsF1SIUrotWe4PKs2g1Q==->0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w== -->
 
 FDFCHhtkO3OemRIzHw33P4kgHee3S+ygHBLHkm1cQhLoID9W8OgvLBaSAdy4aJd+y3zsF1SIUrotWe4PKs2g1Q==->0jDFvJz3EipSZQ/RSH8b33gSCyPyUZJ7oswTslasoIUgiCM6QtjZhQoRz/LR/8y8/I80DBACzb7JUfGrrPMc3w== 
 
 
 prior 
 
<!-- 5mYiI6iFuquTynNDpGcPDAsIKDcWctWKVfMT9N1rTATrts7xhDtKqPk4x3I43Urkn+D5xzYuQkEIqPJMehUT/Q== -->
 
 5mYiI6iFuquTynNDpGcPDAsIKDcWctWKVfMT9N1rTATrts7xhDtKqPk4x3I43Urkn+D5xzYuQkEIqPJMehUT/Q== 
 
 Blog.Site.Name 
 
 value 
 
 My Journal 
 
 
<!-- 5mYiI6iFuquTynNDpGcPDAsIKDcWctWKVfMT9N1rTATrts7xhDtKqPk4x3I43Urkn+D5xzYuQkEIqPJMehUT/Q==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== -->
 
 5mYiI6iFuquTynNDpGcPDAsIKDcWctWKVfMT9N1rTATrts7xhDtKqPk4x3I43Urkn+D5xzYuQkEIqPJMehUT/Q==->5HzpbSouqIEh8jD2za1+kpxBaYSDPrJcpyUThGr9RGbl0J8l7MpLB1c9cGg7ORvAeTftEFHrs1bsF+s+TKSgeQ== 
 
 
 site 
 
<!-- 5mYiI6iFuquTynNDpGcPDAsIKDcWctWKVfMT9N1rTATrts7xhDtKqPk4x3I43Urkn+D5xzYuQkEIqPJMehUT/Q==->FDFCHhtkO3OemRIzHw33P4kgHee3S+ygHBLHkm1cQhLoID9W8OgvLBaSAdy4aJd+y3zsF1SIUrotWe4PKs2g1Q== -->
 
 5mYiI6iFuquTynNDpGcPDAsIKDcWctWKVfMT9N1rTATrts7xhDtKqPk4x3I43Urkn+D5xzYuQkEIqPJMehUT/Q==->FDFCHhtkO3OemRIzHw33P4kgHee3S+ygHBLHkm1cQhLoID9W8OgvLBaSAdy4aJd+y3zsF1SIUrotWe4PKs2g1Q== 
 
 
 prior

To find the current name of a site, we look for site names that have not been replaced.
Let's take this in two parts.

First, look for all of the names of a site.
This will include all past values.

In [23]:
var namesOfSite = Given<Site>.Match((site, facts) =>
    from name in facts.OfType<SiteName>()
    where name.site == site
    select name);

var names = await jinagaClient.Query(namesOfSite, site);

names.Count()

3

Second, let's filter this history.
We only want the names for which there is no next value.

In [24]:
namesOfSite = Given<Site>.Match((site, facts) =>
    from name in facts.OfType<SiteName>()
    where name.site == site
    // Filter out names for which a next name exists
    where !facts.Any<SiteName>(next => next.prior.Contains(name))
    select name);

names = await jinagaClient.Query(namesOfSite, site);

names.Count()

1

In [25]:
names.Single().value

My Journal